In [ ]:
!pip install -y tf-nightly

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import random

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from tensorflow.keras.layers import Dropout, BatchNormalization, Dense
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint,EarlyStopping

from tensorflow.keras import mixed_precision



In [ ]:
data_dir = './Flowers299'
base_dir = '.\Flowers_split' 
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

In [ ]:
Name=[]
for file in os.listdir(data_dir):
    Name+=[file]
    
print(Name)
print(len(Name))

In [ ]:
labels_series=pd.Series(Name)

labels_series.to_csv('labels1.txt',index=False,header=False)

In [ ]:
import re

def add_space_before_capitals(input_file, output_file):
    try:
        # Read the content of the input file
        with open(input_file, 'r') as file:
            content = file.read()

        # Use regular expression to add a space before each capital letter
        modified_content = re.sub(r'(?<!^)(?<!\s)(?=[A-Z])', ' ', content)

        # Write the modified content to the output file
        with open(output_file, 'w') as file:
            file.write(modified_content)

        print(f"Processed content has been saved to '{output_file}'.")

    except FileNotFoundError:
        print(f"The file '{input_file}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
input_file = 'labels1.txt'  # Replace with your input file name
output_file = 'labels.txt'  # Replace with your desired output file name
add_space_before_capitals(input_file, output_file)


In [ ]:
N=[]
for i in range(len(Name)):
    N+=[i]
    
normal_mapping=dict(zip(Name,N)) 
reverse_mapping=dict(zip(N,Name)) 

def mapper(value):
    return reverse_mapping[value]

In [ ]:
dataset=[]
count=0
for file in tqdm(os.listdir(data_dir)):
    path=os.path.join(data_dir,file)
    for im in os.listdir(path):
#         image=load_img(os.path.join(path,im), color_mode='rgb', target_size=(150,150))
#         image=img_to_array(image)
#         image=image/255.0
#         dataset+=[[image,count]]
          count=count+1 # count in first for only
n = count
print(n)

In [ ]:
n=len(dataset)
print(n)

In [ ]:
num=[]
for i in range(n):
    num+=[i]
random.shuffle(num)
print(num[0:5])

In [ ]:
#data,labels=zip(*dataset)
#data=np.array(data)
#labels=np.array(labels)

for i in tqdm(range(n)):
    dataset[i][0]=dataset[i][0].astype(np.float16)

train = np.array([dataset[i][0] for i in num[0:(n//10)*8]])
trainlabel = np.array([dataset[i][1] for i in num[0:(n//10)*8]])

test = np.array([dataset[i][0] for i in num[(n//10)*8:]])
testlabel = np.array([dataset[i][1] for i in num[(n//10)*8:]])

#train=data[num[0:(n//10)*8]]
#trainlabel=labels[num[0:(n//10)*8]]

# test=data[num[(n//10)*8:]]
# testlabel=labels[num[(n//10)*8:]]

In [ ]:
trainlabel2=to_categorical(trainlabel)

In [ ]:
datagen = ImageDataGenerator(rescale=1.0/255.0)
datagen2 = ImageDataGenerator(rescale=1.0/255.0,horizontal_flip=True,vertical_flip=True,rotation_range=20,zoom_range=0.2,
                       )

train_generator = datagen2.flow_from_directory(
    directory=train_dir,  # Use the new train directory
    target_size=(224, 224),
    batch_size=8,
    color_mode='rgb',
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    directory=val_dir,  # Use the new validation directory
    target_size=(224, 224),
    batch_size=8,color_mode='rgb',
    class_mode='categorical'
)

test_generator = datagen.flow_from_directory(
    directory=test_dir,  # Use the new test directory
    target_size=(224, 224),
    batch_size=8,color_mode='rgb',
    class_mode='categorical',
    shuffle=False
)

In [ ]:
print(trainx.shape)
print(testx.shape)
print(trainy.shape)
print(testy.shape)

In [ ]:
datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=20,zoom_range=0.2,
                        width_shift_range=0.2,height_shift_range=0.2,shear_range=0.1,fill_mode="reflect")

In [ ]:
pretrained_model3 = tf.keras.applications.MobileNet(input_shape=(224,224,3),include_top=False,weights='imagenet',pooling='avg')
pretrained_model3.trainable = False

In [ ]:
inputs3 = pretrained_model3.input
x3 = tf.keras.layers.Dense(128, activation='relu')(pretrained_model3.output)
x3 = BatchNormalization()(x3)
x3 = Dropout(0.5)(x3)
outputs3 = tf.keras.layers.Dense(299, activation='softmax')(x3)
model = tf.keras.Model(inputs=inputs3, outputs=outputs3)

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])


In [ ]:
from tensorflow.keras.optimizers import Adam # imports Adam optimizer
model.compile(optimizer=Adam(learning_rate=0.004),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
checkpoint_callback = ModelCheckpoint(
    filepath='new_best_model.keras',       # Path where the model will be saved
    monitor='val_loss',             # Metric to monitor (you can use 'val_accuracy' as well)
    save_best_only=True,            # Save only the best model based on the monitored metric
    mode='min',                     # 'min' to save model when validation loss decreases
    verbose=1                       # Prints messages when the model is saved
)
early_stopping = EarlyStopping(
    monitor='val_loss',         # Metric to monitor (you can use 'val_accuracy')
    patience=3,                 # Number of epochs with no improvement after which training will stop
    restore_best_weights=True,  # Restore model weights from the epoch with the best performance
    verbose=1                   # Print a message when stopping
)


In [ ]:
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
checkpoint_callback = ModelCheckpoint(
    filepath='best_model.keras',       # Path where the model will be saved
    monitor='val_loss',             # Metric to monitor (you can use 'val_accuracy' as well)
    save_best_only=True,            # Save only the best model based on the monitored metric
    mode='min',                     # 'min' to save model when validation loss decreases
    verbose=1                       # Prints messages when the model is saved
)
early_stopping = EarlyStopping(
    monitor='val_loss',         # Metric to monitor (you can use 'val_accuracy')
    patience=3,                 # Number of epochs with no improvement after which training will stop
    restore_best_weights=True,  # Restore model weights from the epoch with the best performance
    verbose=1                   # Print a message when stopping
)


In [ ]:
his=model.fit(train_generator,             # Training data generator
    validation_data=validation_generator
              ,epochs=10, callbacks=[lr_schedule,checkpoint_callback,early_stopping] )

In [ ]:
testx=test_generator

In [ ]:
y_pred=model.predict(testx)


In [ ]:
pred=np.argmax(y_pred,axis=1)
ground = test_generator.classes
print(classification_report(ground,pred,target_names=list(test_generator.class_indices.keys())))

In [ ]:
get_acc = his.history['accuracy']
value_acc = his.history['val_accuracy']
get_loss = his.history['loss']
validation_loss = his.history['val_loss']

epochs = range(len(get_acc))
plt.plot(epochs, get_acc, 'r', label='Accuracy of Training data')
plt.plot(epochs, value_acc, 'b', label='Accuracy of Validation data')
plt.title('Training vs validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
epochs = range(len(get_loss))
plt.plot(epochs, get_loss, 'r', label='Loss of Training data')
plt.plot(epochs, validation_loss, 'b', label='Loss of Validation data')
plt.title('Training vs validation loss')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
img0_path='Flowers299/Wandflower/0acddc9c05.jpg'
load_img(img0_path,target_size=(224,224))

In [ ]:
image=load_img(img0_path,target_size=(224,224))
image=img_to_array(image) 
image=image/255.0
prediction_image=np.array(image)
prediction_image= np.expand_dims(image, axis=0)

In [ ]:
prediction=model.predict(prediction_image)
value=np.argmax(prediction)
name=mapper(value)
print("Prediction is {}.".format(name))

In [ ]:
pred2=model.predict(test)
print(test.shape)
print(pred2.shape)

PRED=[]
for item in pred2:
    value2=np.argmax(item)      
    PRED+=[value2]

In [ ]:
ANS=testlabel

In [ ]:
accuracy=accuracy_score(ANS,PRED)
print(accuracy)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load your FP16 model
model = load_model('best_model(7).keras')
inputs = tf.keras.Input(shape=model.input_shape[1:], dtype=tf.float32, name='input_fp32')

# Cast the input to FP16 for compatibility with the rest of the model
casted_inputs = tf.keras.layers.Lambda(lambda x: tf.cast(x, dtype=tf.float16))(inputs)

# Pass the casted inputs through the original FP16 model
outputs = model(casted_inputs)

# Create a new model that accepts FP32 input and casts to FP16 internally
model_fp32_input = tf.keras.Model(inputs=inputs, outputs=outputs)
tf.saved_model.save(model, 'my_model')  # This will create a directory 'my_model'
####
model = tf.saved_model.load('my_model')
model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY].inputs[0].set_shape([1, 224, 224, 3])
tf.saved_model.save(model, 'my_model3', signatures=model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY])
# Convert
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir='my_model2', signature_keys=['serving_default'])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter.target_spec.supported_types = [tf.float32]
converter.allow_custom_ops = True
tflite_model = converter.convert()
# Save the converted model
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)
print("Model successfully converted and saved as model.tflite")
####


In [ ]:
####
model = tf.saved_model.load('my_model')
model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY].inputs[0].set_shape([1, 224, 224, 3])
tf.saved_model.save(model, 'my_model2', signatures=model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY])
# Convert
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir='my_model2', signature_keys=['serving_default'])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()
# Save the converted model
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)
print("Model successfully converted and saved as model.tflite")
####

In [ ]:
 converter = tf.lite.TFLiteConverter.from_saved_model('my_model')  # Point to SavedModel folder
    tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
        f.write(tflite_model)

In [ ]:
# !pip uninstall tensorflow
!pip uninstall -y tensorflow 
!pip uninstall -y tf-nightly
!pip cache purge
!pip install tensorflow==2.17.0



In [ ]:
!python -m pip uninstall tensorflow
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

In [ ]:
!pip uninstall -y tf-nightly

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import random

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from tensorflow.keras.layers import Dropout, BatchNormalization, Dense
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint,EarlyStopping

from tensorflow.keras import mixed_precision



In [ ]:
data_dir = './Flowers299'
base_dir = '.\Flowers_split' 
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

In [ ]:
Name=[]
for file in os.listdir(data_dir):
    Name+=[file]
    
print(Name)
print(len(Name))

In [ ]:
labels_series=pd.Series(Name)

labels_series.to_csv('labels1.txt',index=False,header=False)

In [ ]:
import re

def add_space_before_capitals(input_file, output_file):
    try:
        # Read the content of the input file
        with open(input_file, 'r') as file:
            content = file.read()

        # Use regular expression to add a space before each capital letter
        modified_content = re.sub(r'(?<!^)(?<!\s)(?=[A-Z])', ' ', content)

        # Write the modified content to the output file
        with open(output_file, 'w') as file:
            file.write(modified_content)

        print(f"Processed content has been saved to '{output_file}'.")

    except FileNotFoundError:
        print(f"The file '{input_file}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
input_file = 'labels1.txt'  # Replace with your input file name
output_file = 'labels.txt'  # Replace with your desired output file name
add_space_before_capitals(input_file, output_file)


In [ ]:
N=[]
for i in range(len(Name)):
    N+=[i]
    
normal_mapping=dict(zip(Name,N)) 
reverse_mapping=dict(zip(N,Name)) 

def mapper(value):
    return reverse_mapping[value]

In [ ]:
dataset=[]
count=0
for file in tqdm(os.listdir(data_dir)):
    path=os.path.join(data_dir,file)
    for im in os.listdir(path):
#         image=load_img(os.path.join(path,im), color_mode='rgb', target_size=(150,150))
#         image=img_to_array(image)
#         image=image/255.0
#         dataset+=[[image,count]]
          count=count+1 # count in first for only
n = count
print(n)

In [ ]:
n=len(dataset)
print(n)

In [ ]:
num=[]
for i in range(n):
    num+=[i]
random.shuffle(num)
print(num[0:5])

In [ ]:
data,labels=zip(*dataset)
data=np.array(data)
labels=np.array(labels)

for i in tqdm(range(n)):
    dataset[i][0]=dataset[i][0].astype(np.float16)

train = np.array([dataset[i][0] for i in num[0:(n//10)*8]])
trainlabel = np.array([dataset[i][1] for i in num[0:(n//10)*8]])

test = np.array([dataset[i][0] for i in num[(n//10)*8:]])
testlabel = np.array([dataset[i][1] for i in num[(n//10)*8:]])



In [ ]:
datagen = ImageDataGenerator(rescale=1.0/255.0)
datagen2 = ImageDataGenerator(rescale=1.0/255.0,horizontal_flip=True,vertical_flip=True,rotation_range=20,zoom_range=0.2,)

train_generator = datagen2.flow_from_directory(
    directory=train_dir,  # Use the new train directory
    target_size=(224, 224),
    batch_size=8,
    color_mode='rgb',
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    directory=val_dir,  # Use the new validation directory
    target_size=(224, 224),
    batch_size=8,color_mode='rgb',
    class_mode='categorical'
)

test_generator = datagen.flow_from_directory(
    directory=test_dir,  # Use the new test directory
    target_size=(224, 224),
    batch_size=8,color_mode='rgb',
    class_mode='categorical',
    shuffle=False
)

In [ ]:
datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=20,zoom_range=0.2,
                        width_shift_range=0.2,height_shift_range=0.2,shear_range=0.1,fill_mode="reflect")

In [ ]:
pretrained_model3 = tf.keras.applications.MobileNet(input_shape=(224,224,3),include_top=False,weights='imagenet',pooling='avg')
pretrained_model3.trainable = False

In [ ]:
inputs3 = pretrained_model3.input
x3 = tf.keras.layers.Dense(128, activation='relu')(pretrained_model3.output)
x3 = BatchNormalization()(x3)
x3 = Dropout(0.5)(x3)
outputs3 = tf.keras.layers.Dense(299, activation='softmax')(x3)
model = tf.keras.Model(inputs=inputs3, outputs=outputs3)

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])


In [ ]:
from tensorflow.keras.optimizers import Adam # imports Adam optimizer
model.compile(optimizer=Adam(learning_rate=0.004),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
checkpoint_callback = ModelCheckpoint(
    filepath='new_best_model.keras',       # Path where the model will be saved
    monitor='val_loss',             # Metric to monitor (you can use 'val_accuracy' as well)
    save_best_only=True,            # Save only the best model based on the monitored metric
    mode='min',                     # 'min' to save model when validation loss decreases
    verbose=1                       # Prints messages when the model is saved
)
early_stopping = EarlyStopping(
    monitor='val_loss',         # Metric to monitor (you can use 'val_accuracy')
    patience=3,                 # Number of epochs with no improvement after which training will stop
    restore_best_weights=True,  # Restore model weights from the epoch with the best performance
    verbose=1                   # Print a message when stopping
)


In [ ]:
his=model.fit(train_generator,             # Training data generator
    validation_data=validation_generator
              ,epochs=10, callbacks=[lr_schedule,checkpoint_callback,early_stopping] )

In [ ]:
get_acc = his.history['accuracy']
value_acc = his.history['val_accuracy']
get_loss = his.history['loss']
validation_loss = his.history['val_loss']

epochs = range(len(get_acc))
plt.plot(epochs, get_acc, 'r', label='Accuracy of Training data')
plt.plot(epochs, value_acc, 'b', label='Accuracy of Validation data')
plt.title('Training vs validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
epochs = range(len(get_loss))
plt.plot(epochs, get_loss, 'r', label='Loss of Training data')
plt.plot(epochs, validation_loss, 'b', label='Loss of Validation data')
plt.title('Training vs validation loss')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
img0_path='Flowers299/Wandflower/0acddc9c05.jpg'
load_img(img0_path,target_size=(224,224))

In [ ]:
image=load_img(img0_path,target_size=(224,224))
image=img_to_array(image) 
image=image/255.0
prediction_image=np.array(image)
prediction_image= np.expand_dims(image, axis=0)

In [ ]:
prediction=model.predict(prediction_image)
value=np.argmax(prediction)
name=mapper(value)
print("Prediction is {}.".format(name))

In [ ]:
pred2=model.predict(test)
print(test.shape)
print(pred2.shape)

PRED=[]
for item in pred2:
    value2=np.argmax(item)      
    PRED+=[value2]

In [ ]:
ANS=testlabel

In [ ]:
accuracy=accuracy_score(ANS,PRED)
print(accuracy)

In [ ]:
# Load your FP16 model
model = load_model('best_model(7).keras')
inputs = tf.keras.Input(shape=model.input_shape[1:], dtype=tf.float32, name='input_fp32')

# Cast the input to FP16 for compatibility with the rest of the model
casted_inputs = tf.keras.layers.Lambda(lambda x: tf.cast(x, dtype=tf.float16))(inputs)

# Pass the casted inputs through the original FP16 model
outputs = model(casted_inputs)

# Create a new model that accepts FP32 input and casts to FP16 internally
model_fp32_input = tf.keras.Model(inputs=inputs, outputs=outputs)
tf.saved_model.save(model, 'my_model')  # This will create a directory 'my_model'
####
model = tf.saved_model.load('my_model')
model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY].inputs[0].set_shape([1, 224, 224, 3])
tf.saved_model.save(model, 'my_model3', signatures=model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY])
# Convert
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir='my_model2', signature_keys=['serving_default'])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter.target_spec.supported_types = [tf.float32]
converter.allow_custom_ops = True
tflite_model = converter.convert()
# Save the converted model
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)
print("Model successfully converted and saved as model.tflite")
